<a href="https://colab.research.google.com/github/vanha2301/AIR-absa-rt/blob/main/lexicon_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lexicon-based Sentiment (VI) — VietSentiWordNet + NegDict + SacThaiDict

Notebook **cơ bản** (không argparse / không CLI).  
Mục tiêu: đọc 3 file `.txt` bạn đưa và tính sentiment theo luật:

- **VietSentiWordnet_Ver1.3.5.txt** → điểm từ (pos - neg)
- **NegDict.txt** → từ phủ định (không, chẳng, chưa, ...)
- **SacThaiDict.txt** → từ nhấn mạnh / mức độ (rất, quá, cực_kỳ, ...)

> Lưu ý: `underthesea.word_tokenize(format="text")` sẽ nối từ ghép bằng dấu `_`
> (ví dụ: `tốt nhất` → `tốt_nhất`). Vì vậy notebook chuẩn hoá lexicon để match kiểu token này. citeturn0search4


## 0) Chuẩn bị file

Đặt các file này **cùng thư mục** với notebook:

- `VietSentiWordnet_Ver1.3.5.txt`
- `NegDict.txt`
- `SacThaiDict.txt`

Nếu bạn để chỗ khác thì sửa lại biến `LEXICON_PATH / NEG_PATH / INTENS_PATH` ở cell bên dưới.


In [16]:
# (Tuỳ chọn) Cài thư viện nếu máy bạn chưa có
!pip -q install underthesea pandas


In [17]:
from __future__ import annotations

import re
import unicodedata
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Set, Tuple, Optional

import pandas as pd

try:
    from underthesea import word_tokenize
except Exception:
    word_tokenize = None

# ==== ĐƯỜNG DẪN 3 FILE (sửa ở đây nếu cần) ====
BASE_DIR = Path(".")  # thư mục notebook
LEXICON_PATH = BASE_DIR / "VietSentiWordnet_Ver1.3.5.txt"
NEG_PATH = BASE_DIR / "NegDict.txt"
INTENS_PATH = BASE_DIR / "SacThaiDict.txt"

# ==== normalize / tokenize ====
URL_RE = re.compile(r"http\S+|www\.\S+", flags=re.IGNORECASE)
NON_ALPHA_RE = re.compile(r"[^0-9a-zA-ZÀ-ỹ_ ]+")

def nfc(s: str) -> str:
    return unicodedata.normalize("NFC", s)

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = nfc(text).lower()
    text = URL_RE.sub(" ", text)
    text = text.replace("\n", " ")
    text = NON_ALPHA_RE.sub(" ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def tokenize_vi(text: str) -> List[str]:
    text = normalize_text(text)
    if word_tokenize is None:
        return text.split()
    # underthesea format="text" sẽ nối cụm từ bằng "_"
    return nfc(word_tokenize(text, format="text")).split()


In [18]:
def load_word_list(path: Path) -> Set[str]:
    words: Set[str] = set()
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            w = nfc(line.strip()).lower()
            if not w or w.startswith("#"):
                continue
            # chuẩn hoá "tốt nhất" -> "tốt_nhất" để match tokenization
            w = w.replace(" ", "_")
            words.add(w)
    return words

def load_intensifiers(path: Path) -> Dict[str, float]:
    """
    SacThaiDict.txt thường là: word<tab>weight
    Nhưng cũng có thể là: word weight  (space)
    """
    intens: Dict[str, float] = {}
    with open(path, "r", encoding="utf-8-sig") as f:
        for raw in f:
            line = nfc(raw.strip())
            if not line or line.startswith("#"):
                continue
            parts = re.split(r"\s+", line)
            if len(parts) < 2:
                continue
            word = parts[0].lower().replace(" ", "_")
            try:
                weight = float(parts[1].replace(",", "."))
            except ValueError:
                continue
            intens[word] = weight
    return intens

def _parse_synset_terms(synset_terms: str) -> List[str]:
    """
    SynsetTerms ví dụ: "tốt_hơn#3 tốt nhất#2"
    -> cần tách theo mẫu kết thúc "#<digits>"
    """
    synset_terms = nfc(synset_terms.strip())
    return [m.group(1).strip() for m in re.finditer(r"(.+?#\d+)(?=\s+|$)", synset_terms)]

def load_vnsenti_lexicon(path: Path) -> Dict[str, float]:
    """
    Format mỗi dòng (tab-separated):
      POS \t ID \t PosScore \t NegScore \t SynsetTerms \t Gloss
    Trả về: lemma -> avg(PosScore - NegScore)
    """
    lex_sum: Dict[str, float] = defaultdict(float)
    lex_cnt: Dict[str, int] = defaultdict(int)

    with open(path, "r", encoding="utf-8-sig") as f:
        for raw in f:
            line = nfc(raw.strip())
            if not line or line.startswith("#"):
                continue
            parts = line.split("\t")
            if len(parts) < 5:
                continue
            if parts[0].lower() == "pos":  # header (nếu có)
                continue

            try:
                pos_score = float(parts[2].replace(",", "."))
                neg_score = float(parts[3].replace(",", "."))
            except ValueError:
                continue

            score = pos_score - neg_score
            for term in _parse_synset_terms(parts[4]):
                lemma = term.rsplit("#", 1)[0].strip().lower()
                if not lemma:
                    continue
                lemma = nfc(lemma.replace(" ", "_"))
                lex_sum[lemma] += score
                lex_cnt[lemma] += 1

    return {w: lex_sum[w] / lex_cnt[w] for w in lex_sum.keys()}


In [19]:
stopwords = load_word_list("vietnamese-stopwords-dash.txt")

def preprocess(text: str) -> List[str]:
    tokens = tokenize_vi(text)
    out = []
    for t in tokens:
        # giữ negation + intensifier để rule hoạt động
        if t in neg_words or t in intens:
            out.append(t)
            continue
        # lọc stopword
        if t in stopwords:
            continue
        out.append(t)
    return out

# def preprocess(text: str) -> List[str]:
#     tokens = tokenize_vi(text)
#     return [nfc(t).lower().strip() for t in tokens if t.strip()]

def sentiment_score(
    tokens: List[str],
    lexicon: Dict[str, float],
    neg_words: Set[str],
    intens: Dict[str, float],
    neg_window: int = 3,
) -> float:
    total = 0.0
    for i, tok in enumerate(tokens):
        if tok in neg_words:
            continue

        s = lexicon.get(tok)
        if s is None:
            continue

        # intensifier ngay trước
        if i > 0 and tokens[i - 1] in intens:
            s *= intens[tokens[i - 1]]

        # có phủ định trong cửa sổ phía trước
        start = max(0, i - neg_window)
        if any(t in neg_words for t in tokens[start:i]):
            s *= -1

        total += s

    return float(total)

def score_to_label(score: float, pos_th: float = 0.05, neg_th: float = -0.05) -> str:
    if score > pos_th:
        return "positive"
    if score < neg_th:
        return "negative"
    return "neutral"

def predict_sentiment(
    text: str,
    lexicon: Dict[str, float],
    neg_words: Set[str],
    intens: Dict[str, float],
    pos_th: float = 0.05,
    neg_th: float = -0.05,
    neg_window: int = 3,
) -> Tuple[str, float]:
    tokens = preprocess(text)
    sc = sentiment_score(tokens, lexicon, neg_words, intens, neg_window=neg_window)
    return score_to_label(sc, pos_th, neg_th), sc


In [20]:
# ==== Load 3 resources ====
assert LEXICON_PATH.exists(), f"Không thấy file: {LEXICON_PATH}"
assert NEG_PATH.exists(), f"Không thấy file: {NEG_PATH}"
assert INTENS_PATH.exists(), f"Không thấy file: {INTENS_PATH}"

lexicon = load_vnsenti_lexicon(LEXICON_PATH)
neg_words = load_word_list(NEG_PATH)
intens = load_intensifiers(INTENS_PATH)

print("Lexicon size:", len(lexicon))
print("Neg words:", len(neg_words))
print("Intensifiers:", len(intens))

# ==== Demo nhanh ====
examples = [
    "Điện thoại này rất tốt, pin trâu và màn hình đẹp.",
    "Sản phẩm quá tệ, dùng được vài hôm là hỏng.",
    "Tạm ổn, không có gì đặc biệt.",
    "Chất lượng không tốt như mong đợi.",
    "Mình không hề thất vọng, thậm chí rất hài lòng.",
    "Hoàn toàn không đáng tiền, quá thất vọng.",
    "Không tệ như mình nghĩ, dùng cũng ổn.",
]

for s in examples:
    lab, sc = predict_sentiment(s, lexicon, neg_words, intens)
    print(f"- {s} -> {lab:8s} | score={sc:.3f}")


Lexicon size: 1191
Neg words: 11
Intensifiers: 16
- Điện thoại này rất tốt, pin trâu và màn hình đẹp. -> positive | score=0.625
- Sản phẩm quá tệ, dùng được vài hôm là hỏng. -> negative | score=-1.375
- Tạm ổn, không có gì đặc biệt. -> neutral  | score=0.000
- Chất lượng không tốt như mong đợi. -> positive | score=0.500
- Mình không hề thất vọng, thậm chí rất hài lòng. -> positive | score=1.875
- Hoàn toàn không đáng tiền, quá thất vọng. -> positive | score=0.875
- Không tệ như mình nghĩ, dùng cũng ổn. -> positive | score=0.625


In [21]:
from typing import Optional, List
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

def predict_on_csv(
    csv_path: str,
    text_col: str = "text",
    out_path: Optional[str] = None,
    pos_th: float = 0.05,
    neg_th: float = -0.05,
    neg_window: int = 3,
) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    if text_col not in df.columns:
        raise ValueError(f"Missing text column: {text_col}. Available: {list(df.columns)}")

    preds: List[str] = []
    scores: List[float] = []

    for txt in df[text_col].astype(str).tolist():
        lab, sc = predict_sentiment(
            txt, lexicon, neg_words, intens,
            pos_th=pos_th, neg_th=neg_th, neg_window=neg_window
        )
        preds.append(lab)
        scores.append(sc)

    df["pred_label"] = preds
    df["pred_score"] = scores

    # Nếu có label thì in metric (macro-F1 + accuracy)
    if "label" in df.columns:
        y_true = df["label"].astype(str)
        y_pred = df["pred_label"].astype(str)
        print("Accuracy :", f"{accuracy_score(y_true, y_pred):.4f}")
        print("Macro-F1 :", f"{f1_score(y_true, y_pred, average='macro'):.4f}")

    if out_path:
        df.to_csv(out_path, index=False, encoding="utf-8-sig")
        print("Saved:", out_path)

    return df

# Ví dụ (chạy đúng file data_graded.csv):
df_pred = predict_on_csv("data_graded.csv", text_col="text", out_path="data_graded.pred.csv")
df_pred.head()


Accuracy : 0.5745
Macro-F1 : 0.5761
Saved: data_graded.pred.csv


,id,difficulty,text,label,pred_label,pred_score
0,1,easy,"Đáng tiền, pin trâu, màn hình đẹp. Mình sẽ cân...",positive,positive,0.625
1,2,easy,"Giao hàng nhanh, đóng gói cẩn thận. Nếu cải th...",positive,positive,0.500
2,3,easy,"Sản phẩm rất tốt, dùng ổn định.",positive,positive,0.750
3,4,easy,"Âm thanh hay, thiết kế đẹp.",positive,positive,0.625
4,5,easy,"Đáng tiền, pin trâu, màn hình đẹp. Không biết ...",positive,positive,0.625
